My solution for Analytics Vidhya practice hackathon [Identify The Apparel](https://datahack.analyticsvidhya.com/contest/practice-problem-identify-the-apparels/). 

## Import Libraries

In [1]:
from fastai.vision.all import *
from fastai.vision.widgets import *
from minivggnet import *

## Import Dataset

In [2]:
ls ../data/apparel

models/  test/  test.csv*  train/  train.csv*


In [3]:
path = Path("../data/apparel")
Path.BASE_PATH  = None

In [4]:
path.ls()

(#6) [Path('../data/apparel/.ipynb_checkpoints'),Path('../data/apparel/models'),Path('../data/apparel/test'),Path('../data/apparel/test.csv'),Path('../data/apparel/train'),Path('../data/apparel/train.csv')]

In [5]:
(path/"train").ls()

(#60000) [Path('../data/apparel/train/1.png'),Path('../data/apparel/train/10.png'),Path('../data/apparel/train/100.png'),Path('../data/apparel/train/1000.png'),Path('../data/apparel/train/10000.png'),Path('../data/apparel/train/10001.png'),Path('../data/apparel/train/10002.png'),Path('../data/apparel/train/10003.png'),Path('../data/apparel/train/10004.png'),Path('../data/apparel/train/10005.png')...]

In [6]:
train_df = pd.read_csv(path/"train.csv")
train_df['id'] = train_df['id'].apply(lambda x: 'train/'+str(x))
train_df.head()

,id,label
0,train/1,9
1,train/2,0
2,train/3,0
3,train/4,3
4,train/5,0


In [7]:
test_df = pd.read_csv(path/"test.csv")
test_df['id'] = test_df['id'].apply(lambda x: 'test/'+str(x))
test_df.head()

,id
0,test/60001
1,test/60002
2,test/60003
3,test/60004
4,test/60005


## Full Training - MiniVGGnet

In [8]:
def get_x(r): return Path(str(path/r['id'])+'.png')
def get_y(r): return r['label']
apparel = DataBlock(
    blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
    splitter=RandomSplitter(valid_pct=0.00000001),
    get_x= get_x,
    get_y=get_y,
    item_tfms = Resize(size=[28,28]),
    batch_tfms = None
)

In [9]:
dls = apparel.dataloaders(train_df, bs=128)
test_dls= dls.test_dl(test_df)

In [10]:
vgg_learner = Learner(
    dls = dls, 
    model=MiniVGGNet(batch_norm=True, size=28, n_class=dls.c), 
    pretrained=False, 
    loss_func=F.cross_entropy, 
    model_dir=path/"models",
    metrics=accuracy)

In [11]:
vgg_learner.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.382078,None,None,00:46
1,0.333816,None,None,00:48
2,0.308210,None,None,00:47
3,0.263027,None,None,00:48
4,0.242905,None,None,00:46
5,0.207888,None,None,00:48
6,0.177964,None,None,00:47
7,0.144030,None,None,00:48
8,0.111531,None,None,00:47
9,0.099558,None,None,00:47


/home/aaagraw/anaconda3/envs/fastai/lib/python3.8/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [58]:
vgg_learner.export(path/'models/stg1-minivgg.pkl')

In [12]:
predictions = vgg_learner.get_preds(dl=test_dls)
sub_data = test_dls.items.reset_index(drop=True).copy()
sub_data['id'] = sub_data['id'].apply(lambda x: x.replace('test/',''))
sub_data['label'] = np.argmax(np.array(predictions[0]), axis=1) 
sub_data.to_csv('./submission/sub_stg1minivgg_scratch_full.csv', index=False)

In [13]:
vgg_learner.fit_one_cycle(4, lr_max=slice(1e-6,1e-4))

epoch,train_loss,valid_loss,accuracy,time
0,0.103360,None,None,00:46
1,0.106171,None,None,00:51
2,0.101974,None,None,00:48
3,0.096003,None,None,00:46


In [15]:
vgg_learner.fit_one_cycle(5, lr_max=1e-5)

epoch,train_loss,valid_loss,accuracy,time
0,0.099537,None,None,00:46
1,0.100619,None,None,00:48
2,0.101939,None,None,00:48
3,0.102386,None,None,00:47
4,0.095497,None,None,00:48


In [17]:
vgg_learner.fit_one_cycle(5, lr_max=1e-5)

epoch,train_loss,valid_loss,accuracy,time
0,0.104006,None,None,00:48
1,0.096935,None,None,00:48
2,0.095320,None,None,00:48
3,0.099648,None,None,00:47
4,0.095594,None,None,00:47


In [18]:
predictions = vgg_learner.get_preds(dl=test_dls)
sub_data = test_dls.items.reset_index(drop=True).copy()
sub_data['id'] = sub_data['id'].apply(lambda x: x.replace('test/',''))
sub_data['label'] = np.argmax(np.array(predictions[0]), axis=1) 
sub_data.to_csv('./submission/sub_stg2minivgg_scratch_full_further.csv', index=False)